 Copyright © Sorbonne University.

 This source code is licensed under the MIT license found in the
 LICENSE file in the root directory of this source tree.

# Outlook

In this notebook, you will code a naive actor-critic algorithm in the tabular case. Then you will tune it using grid search and Bayesian optimization, potentially using the [optuna](https://optuna.readthedocs.io/en/stable/) library.
Finally, you will get the best hyper-parameters obtained with both methods and perform a statistical test to see if there is a statistically significant difference between these methods and with respect to naive hyper-parameter values.

在本笔记本中，您将实现一个简单的演员-评论家算法（naive actor-critic algorithm）在表格形式下的代码。接着，您将使用网格搜索（grid search）和贝叶斯优化（Bayesian optimization）来调整该算法的参数，可能会使用 optuna 库。最后，您将获得两种方法得到的最佳超参数，并进行统计测试，以查看这两种方法之间是否存在统计显著差异，以及与简单超参数值相比的情况。

## Install libraries

In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install 'easypip>=1.2.0'")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("swig")
easyinstall("bbrl>=0.2.2")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")
easyinstall("optuna")
easyinstall("gymnasium")
easyinstall("mazemdp")

import numpy as np
import os
from typing import List  # Ensure compatibility with Python 3.8 and below, with the same functionality as 'list'.

import hydra
import optuna
import yaml
from omegaconf import OmegaConf, DictConfig

# For visualization
os.environ["VIDEO_FPS"] = "5"
if not os.path.isdir("./videos"):
    os.mkdir("./videos")

from IPython.display import Video

In [2]:
import torch
import torch.nn as nn

In [60]:
import gymnasium as gym

from bbrl.utils.chrono import Chrono

import matplotlib
import matplotlib.pyplot as plt

from mazemdp.toolbox import sample_categorical
from mazemdp.mdp import Mdp
from bbrl_gymnasium.envs.maze_mdp import MazeMDPEnv
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder
from functools import partial

# matplotlib.use("TkAgg")
matplotlib.use("Agg")

In [4]:
env = gym.make(
    "MazeMDP-v0",
    kwargs={"width": 5, "height": 5, "ratio": 0.2, "hit": 0.0},
    render_mode="human",
)
env.reset()
env.unwrapped.init_draw("The maze")

       [ 1,  4,  7, 12, 16],
       [ 2, -1,  8, …

# Step 1: Coding the naive Actor-critic algorithm 
# 编码简单的演员-评论家算法

We consider the naive actor-critic algorithm with a categorical policy.
The algorithm learns a critic with the standard temporal difference mechanism
using a learning rate $\alpha_{critic}$.

We consider a value-based critic $V(s)$. The extension to an action value function $Q(s,a)$ is straightforward.

To update the critic, the algorithm computes the temporal difference error:

$$\delta_t = r(s_t, a_t) + \gamma V^{(n)}(s_{t+1})-V^{(n)}(s_t).$$

Then it applies it to the critic:

$$V^{(n+1)}(s_t) = V^{(n)}(s_t) + \alpha_{critic} \delta_t.$$

To update the actor, the general idea is the same, using the temporal difference error with another learning rate $\alpha_{actor}$.

However, naively applying the same learning rule would not ensure that the probabilities of all actions in a state sum to 1.
Besides, when the temporal difference error $\delta_t$ is negative, it may happen that the probability of an action gets negative or null, which raises an issue when applying renormalization.

So, instead of applying the naive rule, we apply the following one:
$$ 
\pi_{temp}(a_t|s_t) =  \begin{cases}
\pi^{(i)}(a_t|s_t) + \alpha_{actor} \delta_t & \mathrm{if } \pi^{(i)}(a_t|s_t) + \alpha_{actor} \delta_t > 10^{-8}\\
10^{-8} & \mathrm{otherwise.} \\
\end{cases}
$$

Then we can apply renormalization so that the probabilities of actions still sum to 1, with
$$
\forall a, \pi^{(i+1)}(a|s_t) = \frac{\pi_{temp}^{(i+1)}(a|s_t)} {\sum_{a'} \pi_{temp}^{(i+1)}(a'|s_t)}
$$ with
$$ 
\pi_{temp}^{(i+1)}(a|s_t) =  \begin{cases}
\pi_{temp}(a|s_t) & \mathrm{if } a = a_t\\
\pi^{(i)}(a|s_t) & \mathrm{otherwise.} \\
\end{cases}
$$

## Exercise 1

### 1. Code the naive actor-critic algorithm as specified above.

Some hints:

- a good idea to build this code it to take inspiration from the code of Q-learning, to add an actor (a categorical policy), both learning rates,
and to take care about the renormalization function.

- for the next steps of this lab, having a function to repeatedly call your actor-critic algorithm and save the learning trajectories and
norms of the value function is a good idea.

In [5]:
# To be completed...

# assert False, 'Not implemented yet'
def naive_actor_critic(
        mdp: MazeMDPEnv,
        alpha_critic: float,
        alpha_actor: float,
        nb_episodes: int = 50,
        timeout: int = 50,
        render: bool = True
)->tuple[np.ndarray, List[float], List[int]]:
    # initialize the state value function and policy pi
    v = np.zeros(mdp.nb_states)
    pi = np.ones((mdp.nb_states,mdp.action_space.n))/mdp.action_space.n

    v_norms = []
    time_list = []

    mdp.timeout = timeout  # set maximum step of each epoch

    if render:
        mdp.init_draw("Naive Actor-Critic")

    for _ in range(nb_episodes):
        x, _ = mdp.reset(uniform=True)
        cpt = 0
        terminated = False
        truncated = False

        while not(terminated or truncated):
            if render:
                mdp.draw_v_pi(v, pi.argmax(axis = 1))

            # choose action from policy
            u = np.random.choice(len(pi[x]), p = pi[x])

            y, r, terminated, truncated, _ = mdp.step(u)

            # update critic(value function)
            delta = r + mdp.gamma*(1-terminated) * v[y] - v[x]
            v[x] = v[x] + alpha_critic * delta

            # update actor (policy)
            pi_temp = np.copy(pi[x])
            pi_temp[u] = max(pi_temp[u]+alpha_actor*delta, 1e-8)

            # normalization
            pi[x] = pi_temp/np.sum(pi_temp)

            x = y
            cpt += 1
            # print(f"Episode {_}, Steps: {cpt}, Value: {v[x]}, Delta: {delta}")

        v_norms.append(np.linalg.norm(v))
        time_list.append(cpt)

    if render:
        mdp.current_state = 0
        mdp.draw_v_pi(v, pi.argmax(axis = 1))
    
    return v, v_norms, time_list

In [6]:
alpha_critic = 0.5
alpha_actor = 0.5

v, v_norms, time_list = naive_actor_critic(env.unwrapped,alpha_critic=alpha_critic, alpha_actor=alpha_actor)

       [ 1,  4,  7, 12, 16],
       [ 2, -1,  8, …

### 2. Provide a plot function

Your plot function should show the evolution through time of number of steps the agent takes to find the reward in the maze.
If your algorithm works, this number of steps should decrease through time.

Your plot function should also show a mean and a standard deviation (or some more advanced statistics) over a collection of learning runs.

In [7]:
# To be completed...

# assert False, 'Not implemented yet'

def plot_steps_evaluation(
        mdp: MazeMDPEnv,
        alpha_critic: float,
        alpha_actor: float,
        nb_episodes: int = 30,
        timeout: int = 50,
        nb_repeats: int = 10,
        render: bool = False
):
    time_lists = []
    for run in range(nb_repeats):
        _, _, time_list = naive_actor_critic(mdp, alpha_critic, alpha_actor, nb_episodes, timeout, render)
        time_lists.append(time_list)
    mean_steps = np.mean(time_lists, axis = 0)
    std_steps = np.std(time_lists,axis = 0)

    plt.figure()
    plt.plot(mean_steps, label = "Mean Steps", color = 'b')
    plt.fill_between(range(nb_episodes), mean_steps-std_steps, mean_steps+std_steps, color = 'b', alpha = 0.2, label = "Standard Deviation")
    plt.xlabel('Episodes')
    plt.ylabel('Steps to find reward')
    plt.title('Steps to Find Reward Over Time')
    plt.legend()
    plt.show()
    plt.savefig("output.png")


## Actor-critic hyper-parameters 超参数

To represent the hyper-parameters of the experiments performed in this notebook, we suggest using the dictionary below.
This dictionary can be read using omegaconf.
Using it is not mandatory.
You can also change the value of hyper-parameters or environment parameters at will.

In [8]:
ac_params = {
    "save_curves": False,
    "save_heatmap": True,
    "mdp": {
        "name": "MazeMDP-v0",
        "width": 5,
        "height": 5,
        "ratio": 0.2,
        "render_mode": "human", # "rgb_array"
        },
        
    "log_dir": "./tmp",
    "video_dir": "./tmp/videos",

    "nb_episodes": 100,
    "timeout": 200,
    "render": False, # True, # 
    "nb_repeats": 5,

    "alpha_critic": 0.5,
    "alpha_actor": 0.5,
    }

### 3. Test your code

Once everything looks OK, save the obtained plot for your lab report

In [9]:
# To be completed...

config = OmegaConf.create(ac_params)
env = gym.make(
        config.mdp.name, 
        kwargs={"width":config.mdp.width, "height": config.mdp.height, "ratio": config.mdp.ratio},
        render_mode = config.mdp.render_mode,
)


plot_steps_evaluation(
  env.unwrapped,
  alpha_critic = config.alpha_critic,
  alpha_actor = config.alpha_actor,
  nb_episodes=config.nb_episodes,
  timeout=config.timeout,
  nb_repeats=config.nb_repeats,
  render=config.render
    )

# assert False, 'Not implemented yet'


# Step 2: Tuning hyper-parameters

In this part, you have to optimize two hyper-parameters of the actor-critic algorithm, namely the actor and critic learning rates.
You have to do so using a simple grid search method and some Bayesian optimization method.
For the latter, we suggest using the default sampler from [optuna](https://optuna.readthedocs.io/en/stable/).
Follow the above link to understand how optuna works.
Note that it also supports grid search and many other hyper-parameters tuning algorithms.

You should make sure that the hyper-parameters tuning algorithms that you compare benefit from the same training budget
We suggest 400 training runs overall for each method,
which means 20 values each for the actor and the critic learning rates in the case of grid search.

## Exercise 2

### 1. Perform hyper-parameters tuning with two algorithms as suggested above.

### 2. Provide a "heatmap" of the norm of the value function given the hyper-parameters, after training for each pair of hyper-parameters.

### 3. Collect the value of the best hyper-parameters found with each algorithm. You will need them for Step 3.

### 4. Include in your report the heatmaps and the best hyper-parameters found for each method.

In [34]:
# grid search methode
import itertools
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def grid_search(
            mdp:MazeMDPEnv,
            alpha_critic_values: np.ndarray,
            alpha_actor_values: np.ndarray,
            nb_episodes: int = 50,
            timeout: int = 50,
            nb_repeats: int = 1,
            render: bool = False,
            plot_heatmap: bool = True
) -> tuple[tuple[float, float], float, list[tuple[float, float, float]]]:
    
    best_v_norm = float('-inf')
    best_params = None
    results = []

    param_grid = list(itertools.product(alpha_critic_values,alpha_actor_values))

    for alpha_critic,alpha_actor in tqdm(param_grid, desc = "Grid Search"):
        v_norms_repeat = []
        for _ in range(nb_repeats):
            v, v_norms, time_list = naive_actor_critic(mdp, alpha_critic, alpha_actor, nb_episodes, timeout, render)
            v_norms_repeat.append(v_norms[-1])
        
        avg_v_norm = np.mean(v_norms_repeat)
        results.append((round(alpha_critic,2), round(alpha_actor,2), avg_v_norm))
        if avg_v_norm > best_v_norm:
            best_v_norm = avg_v_norm
            best_params = (alpha_critic, alpha_actor)
    
    if plot_heatmap:
        plt.clf() 
        df = pd.DataFrame(results, columns=['alpha_critic', 'alpha_actor', 'v_norm'])
        sns.heatmap(df.pivot(index='alpha_critic', columns='alpha_actor', values='v_norm'), annot=False, cmap='viridis')
        plt.title('Heatmap of Value Function Norm With Grid Search')
        plt.xlabel('Alpha Critic')
        plt.ylabel('Alpha Actor')
        plt.show()
        plt.savefig('heatmap_grid.png')

    return best_params, best_v_norm, results
alpha_critic_values = np.linspace(0.1, 1, 20)
alpha_actor_values = np.linspace(0.1, 1, 20)
best_params_grid, best_v_nrom, results_grid = grid_search(env.unwrapped,alpha_critic_values, alpha_actor_values, nb_repeats=1)
print(best_params_grid)
print(best_v_nrom)
# print(results_grid)


Grid Search: 100%|██████████| 400/400 [00:30<00:00, 13.30it/s]


(0.9526315789473684, 0.8105263157894737)
2.8174140391424167


In [30]:
# Bayesian optimization method

def bayesian_optimization(
        mdp: MazeMDPEnv,
        nb_episodes: int = 50,
        timeout: int = 50,
        nb_repeats: int = 1,
        n_trials: int = 50,
        render: bool = False,
        plot_heatmap: bool = True
) -> tuple[tuple[float, float], list[tuple[float, float, float]]]:
    def objective(trial):
        alpha_critic = trial.suggest_float('alpha_critic', 0.1, 1)
        alpha_actor = trial.suggest_float('alpha_actor', 0.1, 1)

        v_norms_repeat = []
        for _ in range(nb_repeats):
            v, v_norms, time_list = naive_actor_critic(mdp, alpha_critic, alpha_actor, nb_episodes, timeout, render)
            v_norms_repeat.append(v_norms[-1])

        avg_v_norm = np.mean(v_norms_repeat)
        return avg_v_norm
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    best_params = (study.best_params['alpha_critic'], study.best_params['alpha_actor'])
    # results = [(trial.params['alpha_critic'], trial.params['alpha_actor'], trial.value) for trial in study.trials]
    results = [(round(trial.params['alpha_critic'], 2), round(trial.params['alpha_actor'], 2), trial.value) for trial in study.trials]

    if plot_heatmap:
        plt.clf()
        df = pd.DataFrame(results, columns=['alpha_critic', 'alpha_actor', 'v_norm'])
        sns.heatmap(df.pivot(index='alpha_critic', columns='alpha_actor', values='v_norm'), annot=True, fmt=".2f", cmap='viridis')
        plt.title('Heatmap of Value Function Norm With Bayesian Optimization')
        plt.xlabel('Alpha Critic')
        plt.ylabel('Alpha Actor')
        plt.show()
        plt.savefig('heatmap_bayesian.png')
    return best_params, results

best_params_bayesian, results_bayesian = bayesian_optimization(env.unwrapped,nb_repeats=1)
print(best_params_bayesian)
# print(results_bayesian)

[I 2024-10-01 20:29:17,306] A new study created in memory with name: no-name-75659367-7dc0-4980-ba3f-874ee9bf6d05
[I 2024-10-01 20:29:17,418] Trial 0 finished with value: 2.5195883402028114 and parameters: {'alpha_critic': 0.9309463095870434, 'alpha_actor': 0.1884931325257185}. Best is trial 0 with value: 2.5195883402028114.
[I 2024-10-01 20:29:17,547] Trial 1 finished with value: 1.4384292599492439 and parameters: {'alpha_critic': 0.11342742401054771, 'alpha_actor': 0.8915151951479235}. Best is trial 0 with value: 2.5195883402028114.
[I 2024-10-01 20:29:17,635] Trial 2 finished with value: 2.115393848372981 and parameters: {'alpha_critic': 0.43389481330318114, 'alpha_actor': 0.6337427509173861}. Best is trial 0 with value: 2.5195883402028114.
[I 2024-10-01 20:29:17,721] Trial 3 finished with value: 2.0421874685550327 and parameters: {'alpha_critic': 0.33665672744794684, 'alpha_actor': 0.8924734189126052}. Best is trial 0 with value: 2.5195883402028114.
[I 2024-10-01 20:29:17,797] Tria

(0.8822643420164508, 0.9342835456875935)


In [35]:
print(best_params_bayesian)
print(np.max(results_bayesian))
print(results_bayesian)

(0.8822643420164508, 0.9342835456875935)
2.8107816104845753
[(0.93, 0.19, 2.5195883402028114), (0.11, 0.89, 1.4384292599492439), (0.43, 0.63, 2.115393848372981), (0.34, 0.89, 2.0421874685550327), (0.35, 0.52, 2.0291122441456575), (0.77, 0.42, 2.5913618390050486), (0.38, 0.74, 2.245279572262997), (0.79, 0.35, 2.5587682252334), (0.79, 0.89, 2.6337396911368827), (0.61, 0.47, 2.3528283157325616), (1.0, 0.99, 2.722994805901077), (0.97, 0.99, 2.192650554027876), (0.81, 0.74, 2.673328333026982), (0.99, 0.71, 2.7968056339819247), (0.99, 0.69, 2.723687836586824), (0.68, 0.66, 2.6685616462943638), (0.88, 0.61, 2.7499559238823847), (0.88, 0.58, 2.429743200462188), (0.5, 0.3, 2.2163221022245962), (0.66, 0.81, 2.446394323261705), (0.86, 0.12, 2.2829481994972496), (0.9, 0.66, 2.623995496684504), (0.97, 0.76, 2.7879352399814454), (0.87, 0.79, 2.705917480597988), (0.72, 0.57, 2.532414918841093), (0.94, 0.81, 2.6895542025492736), (0.22, 0.52, 1.743207498627244), (0.54, 0.61, 2.4787350989276384), (0.84,

# Step 3: Statistical tests

Now you have to compare the performance of the actor-critic algorithm tuned
with all the best hyper-parameters you found before, using statistical tests.

The functions below are provided to run Welch's T-test over learning curves.
They have been adapted from a github repository: https://github.com/flowersteam/rl_stats
You don't need to understand them in detail (though it is always a good idea to try to understand more code).

In [51]:
from scipy.stats import ttest_ind
import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats
from scipy.stats import ttest_ind

In [52]:
def compute_central_tendency_and_error(id_central, id_error, sample):

    try:
        id_error = int(id_error)
    except:
        pass

    if id_central == "mean":
        central = np.nanmean(sample, axis=1) 
    elif id_central == "median":
        central = np.nanmedian(sample, axis=1)
    else:
        raise NotImplementedError

    if isinstance(id_error, int):
        low = np.nanpercentile(sample, q=int((100 - id_error) / 2), axis=1)
        high = np.nanpercentile(sample, q=int(100 - (100 - id_error) / 2), axis=1)
    elif id_error == "std":
        low = central - np.nanstd(sample, axis=1)
        high = central + np.nanstd(sample, axis=1)
    elif id_error == "sem":
        low = central - np.nanstd(sample, axis=1) / np.sqrt(sample.shape[0])
        high = central + np.nanstd(sample, axis=1) / np.sqrt(sample.shape[0])
    else:
        raise NotImplementedError

    return central, low, high

In [53]:
def run_test(test_id, data1, data2, alpha=0.05):
    """
    Compute tests comparing data1 and data2 with confidence level alpha
    :param test_id: (str) refers to what test should be used
    :param data1: (np.ndarray) sample 1
    :param data2: (np.ndarray) sample 2
    :param alpha: (float) confidence level of the test
    :return: (bool) if True, the null hypothesis is rejected
    """
    data1 = data1.squeeze()
    data2 = data2.squeeze()
    n1 = data1.size
    n2 = data2.size

    # perform Welch t-test":
    _, p = ttest_ind(data1, data2, equal_var=False)
    return p < alpha

This last function was adapted for the lab.

In [83]:
def perform_test(perf1, perf2, name1, name2, sample_size=20, downsampling_fact=5, confidence_level=0.01):

    perf1 = perf1.transpose()
    perf2 = perf2.transpose()
    nb_datapoints = perf1.shape[1]
    nb_steps = perf1.shape[0]

    legend = [name1, name2]

    # what do you want to plot ?
    id_central = 'mean' # "median"  # 
    id_error = 80  # (percentiles), also: 'std', 'sem'

    test_id = "Welch t-test"  # recommended
    
    # 从发0到nb_datapoints中随机抽取sample_size个样本
    sample1 = perf1[:, np.random.randint(0, nb_datapoints, sample_size)]
    sample2 = perf2[:, np.random.randint(0, nb_datapoints, sample_size)]

    # 对perf的列进行降采样
    steps = np.arange(0, nb_steps, downsampling_fact)
    sample1 = sample1[steps, :]
    sample2 = sample2[steps, :]

    # test
    sign_diff = np.zeros([len(steps)])
    for i in range(len(steps)):
        sign_diff[i] = run_test(
            test_id, sample1[i, :], sample2[i, :], alpha=confidence_level
        )

    central1, low1, high1 = compute_central_tendency_and_error(
        id_central, id_error, sample1
    )
    central2, low2, high2 = compute_central_tendency_and_error(
        id_central, id_error, sample2
    )

    # plot
    _, ax = plt.subplots(1, 1, figsize=(20, 10))
    lab1 = plt.xlabel("training steps")
    lab2 = plt.ylabel("performance")

    plt.plot(steps, central1, linewidth=10)
    plt.plot(steps, central2, linewidth=10)
    plt.fill_between(steps, low1, high1, alpha=0.3)
    plt.fill_between(steps, low2, high2, alpha=0.3)
    leg = ax.legend(legend, frameon=False)

    # plot significative difference as dots
    idx = np.argwhere(sign_diff == 1)
    y = max(np.nanmax(high1), np.nanmax(high2))
    plt.scatter(steps[idx], y * 1.05 * np.ones([idx.size]), s=100, c="k", marker="o")

    # style
    for line in leg.get_lines():
        line.set_linewidth(10.0)
    ax.spines["top"].set_linewidth(5)
    ax.spines["right"].set_linewidth(5)
    ax.spines["bottom"].set_linewidth(5)
    ax.spines["left"].set_linewidth(5)

    plt.savefig(
        f"./{name1}_{name2}.png", bbox_extra_artists=(leg, lab1, lab2), bbox_inches="tight", dpi=100
    )
    # plt.show()

## Exercise 3

As hyper-parameters, you will use:

- naive tuning, that is a pair (0.5, 0.5) for the actor and critic learning rates,
- the best hyper-parameters you found with the different tuning algorithms you used before.

### 1. For each set of hyper-parameters, collect a large dataset of learning curves.

We suggest using 150 training episodes.

### 2. Perform statistical comparisons

- Take two datasets of learning curves obtained with the hyper-parameters sets that you found with different tuning algorithms.
- Use the ``` perform_test(...)``` function to compare each possible pair of sets.

You should obtain an image for each pair you have tried.
In this image, black dots signal the time step where there is a statistically significant difference between two learning curves.

 ### 3. Conclude.

In [80]:
nb_episodes = 150
nb_repeats = 10

alpha_critic, alpha_actor = (0.5, 0.5)
alpha_critic_grid,alpha_actor_grid = best_params_grid
alpha_critic_bayesian,alpha_actor_bayesian = best_params_bayesian

Lists_1,Lists_2,Lists_3 = [],[],[]
for _ in range(nb_repeats):
    v_1, v_norms_1, time_list_1 = naive_actor_critic(env.unwrapped,alpha_critic=alpha_critic, alpha_actor=alpha_actor, nb_episodes=nb_episodes, render=False)
    v_2, v_norms_2, time_list_2 = naive_actor_critic(env.unwrapped,alpha_critic=alpha_critic_grid, alpha_actor=alpha_actor_grid,nb_episodes=nb_episodes, render=False)
    v_3, v_norms_3, time_list_3 = naive_actor_critic(env.unwrapped,alpha_critic=alpha_critic_bayesian, alpha_actor=alpha_actor_bayesian, nb_episodes=nb_episodes, render=False)
    Lists_1.append(time_list_1)
    Lists_2.append(time_list_2)
    Lists_3.append(time_list_3)


In [84]:
learning_curves = {}

learning_curves["naive"] = np.array(Lists_1)
learning_curves["grid"] = np.array(Lists_2)
learning_curves["bayesian"] = np.array(Lists_3)

comparaison_results = {}
for name1 in learning_curves.keys():
    for name2 in learning_curves.keys():
        if name1 != name2:
            comparaison_results[(name1, name2)] = perform_test(
                learning_curves[name1],
                learning_curves[name2],
                name1 = name1,
                name2 = name2,
                confidence_level=0.05,
            )

# Lab report

Your report should contain:
- your source code (probably this notebook), do not forget to put your names on top of the notebook,
- in a separate pdf file with your names in the name of the file:
    + a detailed enough description of the choices you have made: the parameters you have set, the libraries you have used, etc.,
    + the heatmaps obtained using the hyper-parameters tuning algorithms that you have used,
    + the figures resulting from performing Welch's T-test using the best hyper-parameters from the above approaches,
    + your conclusion from these experiments.

Beyond the elements required in this report, any additional studies will be rewarded.
For instance, you can try using a Q-function as critic, using random search as hyper-parameters tuning algorithm,
using more challenging environments, etc.